https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Preprocess

In [3]:
import pandas as pd

In [5]:
train_df = pd.read_csv('train_dataset.csv')
train_df.head()

,name,class
0,../../../Library/Mobile Documents/com~apple~Cl...,deer
1,../../../Library/Mobile Documents/com~apple~Cl...,cat
2,../../../Library/Mobile Documents/com~apple~Cl...,deer
3,../../../Library/Mobile Documents/com~apple~Cl...,truck
4,../../../Library/Mobile Documents/com~apple~Cl...,automobile


In [6]:
test_df = pd.read_csv('test_dataset.csv')
test_df.head()

,name,class
0,../../../Library/Mobile Documents/com~apple~Cl...,cat
1,../../../Library/Mobile Documents/com~apple~Cl...,horse
2,../../../Library/Mobile Documents/com~apple~Cl...,deer
3,../../../Library/Mobile Documents/com~apple~Cl...,horse
4,../../../Library/Mobile Documents/com~apple~Cl...,automobile


In [7]:
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    width_shift_range = 0.3,
    zoom_range = 0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale= 1./255.
)

In [8]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size= batch_size
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size= batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


## Build Model

In [9]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [12]:
len(train_df) // batch_size

1562

In [14]:
# 모델 학습 시킬때 시간이 오래걸림
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs = num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)